### 네이버 메뉴 추출

- 방법 : 크롬 개발자 도구 이용
- 전체 소스코드에서 위치 찾아(화면 해당 컨텐츠에서 단축메뉴 검사 이용)
- 태그 확인하고 태그 추출
- 동일태그가 여러 개인 경우 class 속성 이용해서 추출

In [2]:
from urllib.request import urlopen
import bs4

In [4]:
url = "https://www.naver.com"

# url에 접속
html = urlopen(url)
type(html)
# html.read() 생략하고 response 객체를 바로 파서기에 적용해도 됨

http.client.HTTPResponse

In [6]:
# 파서에 등록해 obj(객체 인스턴스)로 변환
bs_obj = bs4.BeautifulSoup(html, "html.parser")

In [10]:
# 첫 번째 메뉴 블럭 파싱 (ul class : list_nav type_fix)
ul = bs_obj.find("ul",{"class":"list_nav type_fix"})
print(type(ul)) # 태그 타입 -> ul 이 리스트의 대괄호처럼 인식, li는 ul의 원소처럼 인식
# print(ul)
# 위에서 추출한 ul 객체는 bs4.element.Tag 임
# 이 객체는 집합적 자료의 속성을 갖고 있기 때문에 반복 요소로 사용 가능
for li in ul :
    print(li)

<class 'bs4.element.Tag'>


<li class="nav_item">
<a class="nav" data-clk="svc.mail" href="https://mail.naver.com/"><i class="ico_mail"></i>메일</a>
</li>


<li class="nav_item"><a class="nav" data-clk="svc.cafe" href="https://section.cafe.naver.com/">카페</a></li>


<li class="nav_item"><a class="nav" data-clk="svc.blog" href="https://section.blog.naver.com/">블로그</a></li>


<li class="nav_item"><a class="nav" data-clk="svc.kin" href="https://kin.naver.com/">지식iN</a></li>


<li class="nav_item"><a class="nav shop" data-clk="svc.shopping" href="https://shopping.naver.com/"><span class="blind">쇼핑</span></a></li>


<li class="nav_item"><a class="nav shoplive" data-clk="svc.shoppinglive" href="https://shoppinglive.naver.com/home"><span class="blind">쇼핑LIVE</span></a></li>


<li class="nav_item"><a class="nav" data-clk="svc.pay" href="https://order.pay.naver.com/home">Pay</a></li>


<li class="nav_item">
<a class="nav" data-clk="svc.tvcast" href="https://tv.naver.com/"><i class="ico_tv"></i>TV<

In [11]:
# findAll 함수 사용해서 ul 안의 모든 li를 추출
lis = ul.findAll("li")
print(lis)

[<li class="nav_item">
<a class="nav" data-clk="svc.mail" href="https://mail.naver.com/"><i class="ico_mail"></i>메일</a>
</li>, <li class="nav_item"><a class="nav" data-clk="svc.cafe" href="https://section.cafe.naver.com/">카페</a></li>, <li class="nav_item"><a class="nav" data-clk="svc.blog" href="https://section.blog.naver.com/">블로그</a></li>, <li class="nav_item"><a class="nav" data-clk="svc.kin" href="https://kin.naver.com/">지식iN</a></li>, <li class="nav_item"><a class="nav shop" data-clk="svc.shopping" href="https://shopping.naver.com/"><span class="blind">쇼핑</span></a></li>, <li class="nav_item"><a class="nav shoplive" data-clk="svc.shoppinglive" href="https://shoppinglive.naver.com/home"><span class="blind">쇼핑LIVE</span></a></li>, <li class="nav_item"><a class="nav" data-clk="svc.pay" href="https://order.pay.naver.com/home">Pay</a></li>, <li class="nav_item">
<a class="nav" data-clk="svc.tvcast" href="https://tv.naver.com/"><i class="ico_tv"></i>TV</a>
</li>]


In [14]:
# 각 li 태그 내부의 a 태그 접근 후 텍스트 출력("메일")
for li in lis :
    a_tag = li.find("a")
    print(a_tag.text,end=",")

메일,카페,블로그,지식iN,쇼핑,쇼핑LIVE,Pay,TV,

In [16]:
ul1 = bs_obj.find("ul",{"class" : "list_nav NM_FAVORITE_LIST"})
# ul1
lis = ul1.findAll("li")
for li in lis :
    a_tag = li.find("a")
    print(a_tag.text,end=",")

사전,뉴스,증권,부동산,지도,VIBE,책,웹툰,

### 네이버 뉴스 크롤링

In [17]:
url = "https://news.naver.com/"
# html = urlopen(url) 
# RemoteDisconnected Error -> 프로그램을 통해 자동화된 접근이기 때문에 받아들여지지 않음
# 프로그램을 통해 자동화된 bot으로 생각하고 연결을 끊어버림

RemoteDisconnected: Remote end closed connection without response

- 위 코드 처럼 자동화 봇으로 보고 연결을 끊는 경우에는 데이터 전송시 추가되는 header를 구성해 bot이 아님을 증명해야 함
    - header에는 브라우저를 통한 접속이라는 정보를 포함시킨다
    - 크롬 개발자도구 => NetWork => news.naver.com 선택한 후 오른쪽에 새로나타나는 창에서 header 선택
    - requests headers 목록에서 user-agent 확인
- https://developers.whatismybrowser.com/useragents/explore/software_name/chrome/
    - chrome의 user agent로 최근에 업데이트 됨

In [60]:
# header 전송 ㅡ 딕셔너리로 구성함 ㅡ은 보통 requests 객체를 사용함
import requests

headers = {"User-Agent":
           "Mozilla/5.0(Window NT 10.0;Win64;x64) AppleWebKit/537.36 (KHTML,likeGecko) Chrome/87.0.4280.88 Safari/537.36"} 
# User-Agent 는 고정되어있는 문구임! 항상 사용!

url = "https://news.naver.com"
# headers는 파라미터
res = requests.get(url,headers=headers) # 요청 들어감
html = res.text
html

'\n\n\n\n\n\n\n\n\n\n\n\t\n\t\t\n\t\n\t\n\n\n<!DOCTYPE HTML>\n<html lang="ko">\n<head>\n<meta charset="euc-kr">\n<meta http-equiv="X-UA-Compatible" content="IE=edge">\n<meta name="referrer" contents="always">\n<meta http-equiv="refresh" content="600" />\n<meta name="viewport" content="width=1106" />\n\r\n    \r\n    \r\n        \r\n            \r\n            \r\n            \r\n            \r\n            \r\n        \r\n    \r\n    \r\n    \r\n    \r\n    \r\n    \r\n    \r\n    \r\n    \r\n    \r\n    \r\n    \r\n    \r\n    \r\n    \r\n    \r\n    \r\n\r\n<meta property="og:title"       content="네이버 뉴스">\r\n<meta property="og:type"        content="website">\r\n<meta property="og:url"         content="http://news.naver.com/main/home.naver">\r\n<meta property="og:image"       content="https://ssl.pstatic.net/static.news/image/news/ogtag/navernews_200x200_20160804.png"/>\r\n<meta property="og:description" content="정치, 경제, 사회, 생활/문화, 세계, IT/과학 등 언론사별, 분야별 뉴스 기사 제공">\r\n<meta property="

In [61]:
bs_obj = bs4.BeautifulSoup(html,"html.parser")
# html 소스 확인
# print(bs_obj.prettify())

# 헤드라인 뉴스의 구성 ul, class : hdline_article_list
news_list = bs_obj.find("ul",{"class":"hdline_article_list"})
# news_list

# 모든 li 추출
lis = news_list.findAll("li")

# 뉴스 제목 출력 :
for li in lis :
    a_tag = li.find("a")
    print(a_tag.text.strip())

분당 김밥집 식중독 환자 134명으로 늘어...전날보다 31명 증가
대구, 2020년 3월 이후 최다 코로나19 확진자 발생
윤종인 위원장 “개보위, 데이터경제 시대에 적합한 보호체계 구축”
백신 미접종 60∼74세 누구나 접종 기회…동네의원 예약도 가능
신세계 센텀 릴레이 확진…아이스링크 직원 1명 확진


In [36]:
# 관련기사 링크 list에 저장
link_list=[]
for li in lis :
    a_tag = li.find("a")
    link_list.append("https://news.naver.com"+a_tag["href"]) # 속성값을 찾을 때는 태그명["속성값"] 
print(link_list)
# 전체 url로 수집하기 위해 "https://news.naver.com" 을 앞에 따로 붙여서 리스트에 저장함 (더 효율적)

['https://news.naver.com/main/read.naver?mode=LSD&mid=shm&sid1=102&oid=029&aid=0002690599', 'https://news.naver.com/main/read.naver?mode=LSD&mid=shm&sid1=100&oid=020&aid=0003374226', 'https://news.naver.com/main/read.naver?mode=LSD&mid=shm&sid1=100&oid=001&aid=0012576310', 'https://news.naver.com/main/read.naver?mode=LSD&mid=shm&sid1=101&oid=001&aid=0012575973', 'https://news.naver.com/main/read.naver?mode=LSD&mid=shm&sid1=100&oid=016&aid=0001870350']


In [ ]:
# 연습문제
# 네이버 뉴스 메인페이지의 5개 섹션의 헤드라인 기사의 타이틀과 
# 타이틀에 연결된 링크 주소를 수집해서 df으로 작성

In [62]:
import requests

headers = {"User-Agent":
           "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Mobile Safari/537.36"} 
# User-Agent 는 고정되어있는 문구임! 항상 사용!

url = "https://news.naver.com"
res = requests.get(url,headers=headers) # 요청 들어감
html = res.text
html

'<!doctype html>\n<html lang="ko">\n<head>\n<title id="browserTitleArea">네이버 뉴스</title>\n<script>\n\t(function () {\n\t\ttry {\n\t\t\tif (isAbleApplyPrefersColorScheme()) {\n\t\t\t\t\n                document.querySelector("html").classList.add("DARK_THEME");\n\t\t\t}\n\t\t} catch(e) {}\n\n\t\tfunction isAbleApplyPrefersColorScheme() {\n            \n            if (window.matchMedia("(prefers-color-scheme)").matches === false) {\n                return false;\n            }\n\n\t\t\tvar userAgent = navigator.userAgent;\n\n\t\t\tif (userAgent.indexOf("NAVER") > -1) {\n\t\t\t\t\n\t\t\t\tif (/.*NAVER\\([a-zA-Z]*;\\s[a-zA-Z]*;\\s([0-9]*);/.test(userAgent)) {\n\t\t\t\t\treturn Number(RegExp.$1) >= 1000;\n\t\t\t\t}\n\t\t\t} else {\n\t\t\t\t\n\t\t\t\treturn document.cookie.indexOf("NSCS=1") > -1;\n\t\t\t}\n\n\t\t\treturn false;\n\t\t}\n\t})();\n</script>\n<meta charset="utf-8">\n<meta name="viewport" content="width=device-width,initial-scale=1.0,maximum-scale=1.0,minimum-scale=1.0,user-scala

In [67]:
bs_obj = bs4.BeautifulSoup(html,"html.parser")
# html 소스 확인
# print(bs_obj.prettify())

# 헤드라인 뉴스의 구성 ul, class : hdline_article_list
news_list = bs_obj.find("ul",{"class":"hdline_article_list"})
# news_list

# 모든 li 추출
lis = news_list.findAll("li")

# 뉴스 제목 출력 :
for li in lis :
    a_tag = li.find("a")
    print(a_tag.text.strip())

AttributeError: 'NoneType' object has no attribute 'findAll'

['https://news.naver.com/main/read.naver?mode=LSD&mid=shm&sid1=102&oid=469&aid=0000621770', 'https://news.naver.com/main/read.naver?mode=LSD&mid=shm&sid1=102&oid=047&aid=0002322187', 'https://news.naver.com/main/read.naver?mode=LSD&mid=shm&sid1=105&oid=028&aid=0002555396', 'https://news.naver.com/main/read.naver?mode=LSD&mid=shm&sid1=102&oid=001&aid=0012577664', 'https://news.naver.com/main/read.naver?mode=LSD&mid=shm&sid1=102&oid=001&aid=0012576660']
